In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.fftpack import fft, ifft


def Window(ts):
    '''This function applies a parabolic window to the signal input'''
    n = len(ts) #n is length of time series
    W = np.zeros((n,1)) #initialize empty array
    #create parabola from 0 to length n
    for j in np.arange(0,n):
        W[j] = 1 - np.power((2*(j + 1)/(n+1)-1),2)
    #multiply parabola and signal to get windowed signal (result)
    result = ts * W 
    return result

def Bridge(ts):
    '''This function bridge detrends the signal input ts
    The function subtracts a line connecting the first and last points from the signal'''
    n = len(ts)
    #points is the length of the time series from n-1 to 0 (backwards i.e. n-1, n-2,...0)
    points = np.arange(n-1, -1, -1).reshape(1,n)
    #line is a straight line connecting first and last point
    line = ((ts[0]-ts[n-1])*points/(n-1))+ts[n-1]
    line = line.transpose() #transposes line from row to column array
    #subtract line from time series to return bridge detrended signal (result)
    result = ts - line
    return result 

def Linreg(x,y):
    '''This function calculates the slope and correlation coefficient of a linear fit model
    result[0]: Slope
    result[1]: Correlation coefficient'''
    #calculate mean of x and y (lgfrek and lgPyy when called in FracTool)
    mikro = np.mean(x) #mean of lgfrek
    nu = np.mean(y) #mean of lgPyy
    mikro_sub = x - mikro #subtract lgfrek mean from lgfrek array 
    nu_sub = y - nu #subtract lgPyy mean from lgPyy array
    #multiply mean-subtracted arrays and add sum of elements 
    Qxy = np.sum(mikro_sub*nu_sub)
    Qx = sum(mikro_sub**2) #sum of sqaured differences of lgfrek
    Qy = sum((nu_sub**2)) #sum of squared differences of lgPyy
    result = np.empty([2,1]) #initialize empty result array
    result[0] = Qxy/Qx #slope of linear regression
    result[1] = Qxy/(Qx*Qy)**0.5 #correlation coefficient of linear regression
    
    return result
 
    
def Spec(ts, p, avg, tr): 
    '''This function performs a spectral analysis of the time series
    where ts is the time series, p is the nth power of 2 where 2^n<=length
    of time series. If avg=0, then spectra for all, if avg=1, then spectra avg.
    An array of the following results will be returned.
    result[0]: Hurst coefficient
    result[1]: correlation coefficient
    result[2]: beta from Fourier analysis
    result[3]: Hurst coefficient (high freq excluded)
    result[4]: correlation coefficient (high freq excluded)
    result[5]: beta from Fourier analysis (high freq excluded)'''
    #initialize empty array for results outlined above
    result = np.empty([6,1])
    f = 1/tr
    n = 2**p #p is the greatest integer where 2^p < n 
    
    #compute the estimated power of time series (Pyy):
    
    #compute the discrete fourier transform of time series 
    #Y = scipy.fftpack.fft(ts.flatten()) #time series needs to be 1D (flattened from 2D)
    if len(ts)!= 2**p:
        Y = scipy.fftpack.fft(ts.flatten(),2**(p+1)) #time series needs to be 1D (flattened from 2D)
    elif len(ts) == 2**p:
        Y = scipy.fftpack.fft(ts.flatten(),2**(p))
    Y_conj = Y.conj() #return conjucate of every element in Y, eg. conj() of a+bi is a-bi
    Pyy = Y*Y_conj/n #multiply Y by conj of Y to get absolute value of Y, squared, then divide by length
    Pyy = Pyy[0:int(n/2)] #splice array to only include have first half (it is symmetric)
    Pyy = np.reshape(Pyy, (Pyy.size,1)) #convert Pyy to column array
    
    #clear arrays that are no longer used
    Y = []
    Y_conj = []
    y = []
    
    #generate array of frequencies from 0 to 0.5 (frek)
    frek = f/n*np.arange(1,(n/2)+1)
    frek = np.reshape(frek, (frek.size,1)) #transpose frek to column array 
        
    #if avg=1, then average frek and Pyy before taking spectra
    if avg == 1: 
        #initialize empty 1D arrays
        ff = np.zeros((1,p-1)).flatten()
        pp = np.zeros((1,p-1)).flatten()
        #for loop takes mean of ranges between 2^0 and 2^p
        #output is arrays ff and pp where each element of the array is average of ranges i.e. [avg(2^0-(2^1)-1), avg(2^1-(2^2)-1), ..., avg(2^(p-1)-(2^p)-1)]
        for i in range(1,p):
            #each element in ff is mean of frek from 2^(i-1) to (2^i) - 1 until i = p
            ff[i-1]=np.mean(frek[2**(i-1)-1:2**i-1])
            #each element in pp is mean of Pyy from 2^(i-1) to (2^i) - 1 until i = p
            pp[i-1]=np.mean(Pyy[2**(i-1)-1:2**i-1])
        #clear Pyy and frek and set value to pp and ff omitting first index
        Pyy=[] #clear Pyy
        Pyy = pp[1:len(pp)]
        Pyy = np.reshape(Pyy, (Pyy.size,1)) #convert into 2D array
        frek=[] #clear frek
        frek=ff[1:len(pp)]
        frek = np.reshape(frek,(frek.size,1)) #convert into 2D array
        #now spectra will be of average frek and Pyy values, not all values
      

    #find where Pyy has values of 0, set those values to 0.00001 before taking log
    k = np.where(Pyy==0)
    if len(k)!=0: #if there are values where Pyy = 0
        for kk in range(1,len(k)+1):
            Pyy[k[kk-1]] = 0.00001
            
    #take natural log of Pyy and frek
    lgPyy = np.log(Pyy)
    lgfrek = np.log(frek)
    
    #power spectral density (PSD) plot 
    #plt.title('PSD Before Frequency Filtered')
    #plt.semilogx(frek, Pyy)
    #plt.yscale('log')
    #plt.xlabel('Frequency')
    #plt.ylabel('Power')
    #plt.tight_layout()
    #plt.show()
    
    #compute linear regression of PSD
    #curvefit returns an array of 2 values: (1) slope from lin regression, (2) correlation coeff from lin regression
    curvefit = Linreg(lgfrek,lgPyy)
    #set result values to curvefit results (according to indices outlined in function definition)
    result[2] = curvefit[0,0]*-1  #Beta has opposite sign as slope 
    result[1] = curvefit[1,0]   #Correlation coeff
    #calculate Hurst coefficient based on value of Beta
    if result[2]<1: #if Beta is less than 1, then Hurst coefficient is: 
        result[0] = (result[2]+1)/2 #signal is fGn, H_PSD = (Beta + 1)/2
    elif result[2]>1: #if Beta is greater than 1, then Hurst coefficient is: 
        result[0] = (result[2]-1)/2 #signal is fBm, H_PSD = (Beta - 1)/2
    elif result[2]==1: #if Beta is equal to 1, set Hurst to infinity
        result[0] = inf 
        
    #results 0,1,2 are the Hurst coeff, correlation coeff, and Beta of the raw signal (before high freqs excluded)
    #now will filter frequencies and then compute results 3,4, and 5
    
    #frequency filter:
    #find index of freq 0.01 and 0.10 (or whatever range of freq you want to include)
    min_index1 = np.argmin(abs(frek-0.01)) 
    min_index2 = np.argmin(abs(frek-0.10))
    #splice frek and Pyy arrays to have only values in indices
    frek = frek[min_index1:min_index2]
    Pyy = Pyy[min_index1:min_index2]
    #take log of both arrays
    lgPyy = np.log(Pyy)
    lgfrek = np.log(frek)

    
    #perform linear regression on frequency-filtered arrays
    if avg == 1: 
        curvefit = Linreg(lgfrek, lgPyy)
    else:
        curvefit = Linreg(lgfrek,lgPyy)
    
    #plot of PSD after frequency is filtered
    #plt.title('PSD After Frequency Filtered')
    #plt.semilogx(frek, Pyy)
    #plt.yscale('log')
    #plt.xlabel('Frequency')
    #plt.ylabel('Power')
    #plt.tight_layout()
    #plt.show()
    
    #results of curvefit with high frequencies excluded
    result[5] = curvefit[0,0]*-1 #Beta
    result[4] = curvefit[1,0] #correlation coefficient
    
    #determine Hurst coefficient from Beta found after excluding high frequencies
    if result[5]<1: #if Beta is less than 1
        result[3] = (result[5]+1)/2
    elif result[5]>1: #if Beta is greater than 1
        result[3] = (result[5]-1)/2 #signal is fGn, H_PSD = (Beta + 1)/2
    elif result[5]==1: #if Beta is equal to 1, set Hurst coefficient to infinity 
        result[0] = inf #signal is fBm, H_PSD = (Beta - 1)/2
        
    return result #result is array of the 6 values (results[0] to results[5])

def Stdn(x):
    '''Standard deviation computes the "population" standard deviation, that is, it is 
    normalized by N, where N is the sequence length'''
    mikro = np.mean(x) #mean of time series
    
    n = len(x) #length of time series
    result = (np.sum((x-mikro)**2)/n)**0.5 #standard deviation formula 
   
    return result

def Disper(ts, p):
    '''This function performs dispersional analysis on time series. Dispersion is a measure
    of how much a distribution is stretched or squeezed.
    result[0]: Hurst coefficient
    result[1]: correlation coefficient (r) from linear regression'''
    used = p - 3 #the number of data points used in curve fit, excludes 1/2,1/4,1/8 frequencies
   
    lgsd = np.empty((1,used)).flatten() #initialize empty arrays for for-loop
    lgn = np.empty((1,used)).flatten()
    for i in range(1,used+1): #increment by orders of 2 (scales)
        lgsd[i-1] = np.log(Stdn(ts)) #calculates log of standard deviation of each scale
        lgn[i-1] = np.log(2**i) #natural log of bin size
        tsid = (ts[0:len(ts)-1:2] + ts[1:len(ts):2])/2 #odd time-points + even time-points/2 (avg between points)
        ts = []
        ts = tsid #reset ts to new time-signal which is half the length and the average between points.
        tsid = []
    #lgsd is the natural log of the standard deviation
    #lgn is the natural log of the bin size
    curvefit = Linreg(lgn, lgsd) #compute linear regression 
    result = np.empty((1,2)).flatten() #initialize empty results vector
    result[0] = 1 + curvefit[0] #Hurst coeff - 1 + slope of linear regression
    result[1] = curvefit[1] #correlation coeff of linear regression 
    return result

def Bdswv(ts, p): 
    '''This function performs bridge detrended scaled window variance analysis on an fBm 
    time series'''
    #ignored: column 0 is length, column 1 is smallest bin size, column 2 is largest bin size
    #these are the smallest/largest bins that will be ignored
    #for longer time series (larger p), more bins are ignored
    ignored = np.matrix([[6, 1, 0], [7, 1, 0], [8, 1, 0], [9, 2, 2], [10, 2, 3], [11, 2, 4], [12, 2, 4], [13, 2, 5], [14, 3, 6], [15, 3, 7], [16, 3, 7], [17, 3, 7], [18, 3, 7]])
    #while loop stops once p is equal to a length in column 0
    #smallest and largest are the values in the same row as this length 
    i = 1
    while p >= ignored[i-1,0]:
        smallest = ignored[i-1,1]
        largest = ignored[i-1,2]
        i = i + 1
    #smallest: ignore these smallest bin sizes
    #largest: ignore these largest bin sizes
    #example: p = 10, smallest is 2, largest is 3
    used1 = smallest + 1 #beginning of range of used bins
    used2 = p - largest #end of range of used bins
    n = len(ts)
    #example: p = 10, used1 = 2+1=3, used2 = 10-3 = 7 
     
    #initialize empty arrays for range of used bins    
    lgsd = np.zeros(used2-used1+1)
    lgn = np.zeros(used2-used1+1)
    #for loop goes through every bin in used range (ex bins 3-7 for p =10)
    for i in range(used1, used2 + 1):
        counter = 0
        sd = [] 
        #next for loop runs until n-i, incrementing by 2^i
        #for p = 10, this will be 0 - 1024-2 incrementing by 2^3 for i = 3
        for j in range(0, n-i +1, 2**i): 
            tsmod = ts[j:j+2**i] #splices ts into 2^i splices
            # p = 10, then splice into ts[0,7], ts[8,15] etc 
            # tsmod is a bin (different every time loop runs)
            tsmod = np.reshape(tsmod,(tsmod.size,1)) #make into 2D array
            tsid = Bridge(tsmod) #apply bridge to bin (tsmod)
            tsmod = []
            sd.append(np.std(tsid, ddof=1)) #take standard deviation of bridged bin
            tsid=[]
    
        lgsd[i-used1] = np.log(np.mean(sd)) #natural log of average standard deviation over all bins
        lgn[i-used1] = np.log(2**i) #natural log of bin size
    
    curvefit = Linreg(lgn, lgsd) #perform linear regression on lgn and lgsd
    result = np.empty((1,2)).flatten()
    result[0] = curvefit[0] #slope from linear regression
    result[1] = curvefit[1] #correlation coeff from linear regression

    return result
   
    
def FracTool(ts, tr=1):
    '''This function will aid time series analysis using the concept of statistical fractals.
     This code is adapted from A. Eke, P. Hermán, J. B. Bassingthwaighte, G. M. Raymond, D. B. Percival, 
     M. Cannon, I. Balla, and C. Ikrényi. Physiological time series: distinguishing 
     fractal noises from motions. Pflügers Archiv European Journal of 
     Physiology,4394):403-415, 2000. 
     FracTool determines: 
     1) The signal class of the time series according to the fGn/fBm model
     2) The Hurst coefficients for the fGn or fBm time series
     The input "ts" is a time series of a single column of data points
     The input "tr" is the temporal resolution, which is used to calculate the sampling frequency. It is set
     to 1 as the default parameter'''
    assert (len(ts) !=0), 'time series is empty' #ensures time series is not empty
    #ts = filt_sig(ts,tr) #option to filter signal through Bandpass filter function 
    #initialization 
    H_PSD = -1
    H_Disp = -1
    H_bdSWV = -1
    H_fGn = -1
    H_fBm = -1
    sig_class = None 
    Hurst = -1
    #sig_class = 0: fGn signal
    #sig_class = 1: fGn/fBm boundary
    #sig_class = 2: fBm signal
    #sig_class = 3: outside fGn/fBm model
    
    #parameters 
    n=len(ts) #length of time series input
    #print('Number of raw time-points = ', n)
    i = 1
    while n >= 2**i: #calculates the closest 2^i value in n 
        i += 1
    p = i - 1
    tsid = ts[0:2**p] #splices time series to exclue points after 2^p
    #print('Number of time-points after 2^p splice =', 2**p)
    del ts #delete signal before splice (the raw time series)
    signal_mean = np.mean(tsid) #get mean of spliced time series
    #print('Mean of time series = ', signal_mean)
    n = len(tsid) #update n to length of tsid
    #plt.plot(tsid)
    #plt.show()
    #get Beta value using lowPSDw,e method (w:windowing, e:endmatching, low:excluding high frequencies)
    #apply parabolic window, then bridge detrend the time series
    #calculate spectra estimates excluding frequencies between 1/8 and 1/2
    #Beta is the slope of the regression of the spectral estimates
    result = Spec(Bridge(Window(tsid)),p,0, tr)   
    
    #the value of beta determines the method used to calculate the Hurst coefficient
    if result[5,0]<=1: 
        # if -1<Beta<0.38, then signal is fGn 
        #apply PSD and dispersion method on signal to calculate Hurst coefficient
        H_PSD = (result[5,0]+1)/2 #PSD method
        temp = Disper(tsid,p)
        H_Disp = temp[0] #dispersion method
        H_fGn = (H_PSD+H_Disp)/2 #average of both methods gives final H value
        #H_fGn = H_Disp
        sig_class = 0 #signal is fGn 
        Hurst = H_fGn
        
    elif result[5,0]>1 and result[5,0]<3: 
        #if 1<Beta<3, signal is fBm
        #apply PSD and bdSWV to calculate Hurst, then take average of H values
        H_PSD = (result[5,0]-1)/2 #Hurst calculated by PSD
        temp = Bdswv(tsid,p) 
        H_bdSWV = temp[0] #Hurst calculated by bdSWV
        H_fBm = (H_PSD + H_bdSWV)/2 #avg of both methods giving final Hurst
        sig_class = 2 #signal is fBm
        Hurst = H_bdSWV
        #if Hurst value is below 0.35, it classified the signal wrong, so classify as fGn and find Hurst using fGn method
        if Hurst < 0.30:
            H_PSD = (result[5,0]+1)/2 #PSD method
            temp = Disper(tsid,p)
            H_Disp = temp[0] #dispersion method
            #H_fGn = (H_PSD+H_Disp)/2 #average of both methods gives final H value
            H_fGn = (H_PSD+H_Disp)/2
            sig_class = 0  #are fgn, were classified as fbm
            Hurst = H_fGn
    
    
    else:
        sig_class = 3
        Hurst = None
        
    #if the signal is fBm, omit it 
    if sig_class == 2:
        Hurst = None

    return [sig_class, Hurst]

In [41]:
from fbm import fbm, fgn, times
print('fgn')
for i in range(0,10): 
    fgn_sig = fgn(n=700, hurst = 0.9, length=1, method='daviesharte')
    print(FracTool(fgn_sig))
print('fbm')
for i in range(0,10):
    fbm_sig = fbm(n=700, hurst = 0.3, length=1, method='daviesharte')
    print(FracTool(fbm_sig))

fgn


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: ComplexWarning: Casting complex values to real discards the imaginary part
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: ComplexWarning: Casting complex values to real discards the imaginary part


0.9410284499444368
[0, 1.128128581034575, 1.7237046616168998, -0.6476160349766857]
[0, 0.8537974772963528, 0.7919366505778757, -0.48392547489187987]
0.8988306911696232
[0, 0.926560150428911, 1.1673190130108688, -0.4580426304401371]
[0, 0.919421366207645, 0.971165685512435, -0.41487711494738183]
[0, 0.8589830648688436, 0.7746136777649073, -0.38075008802028304]
1.034012172723115
[2, None, 1.113225996885034, -0.4643419337147153]
[0, 0.8729096305799674, 0.8338154957647763, -0.33435370070007286]
[0, 0.8699179762313386, 0.7922304988914587, -0.438149504128545]
0.9679183013128925
[0, 1.0408566042249885, 1.4667067898389567, -0.6190350110219218]
0.9034852354395141
[0, 0.9098867899241831, 1.0442353696682023, -0.5332714861735117]
fbm
1.3356546342252318
[2, None, 2.06465340292157, -0.7679945991227565]
1.256936117317718
[2, None, 1.134627624348743, -0.37633502568118427]
1.322571656973212
[2, None, 1.4249879424905532, -0.5951904798119525]
1.2987798189401225
[2, None, 1.6719255322106559, -0.6181433929

In [5]:
from fbm import fbm, fgn, times
import numpy as np
#from ipynb.fs.defs.FracTool_Testing import FracTool
import sklearn
from sklearn.metrics import mean_absolute_error
import random as random

def ORIG_6(N1,N2,N3):
    '''This function generates a matrix of known Hurst values and calculates the MSE of the expected Hurst and the 
    actual Hurst outputted by FracTool. The leftmost third of the matrix are fGn signals, the middle third are fBm 
    signals, and the last third are random arrays. 
    N1 = number of rows in matrix
    N2 = number of columns in matrix
    N3 = number of time-points in signal (3rd dimension in matrix)
    '''
    
    #first generate an matrix with known Hurst values and signal class
    
    #create arrays of expected Hurst and class (if interested in class as well)
    Hurst_expected = np.zeros((N1,N2))
    Sig_class_expected = np.zeros((N1,N2))
    Signal_matrix = np.zeros((N1,N2,N3))
    fgn_section = np.floor(N2/2) #end of leftmost third of columns of matrix
   
    for i in range(N1):
        #leftmost third of matrix (fGn signals)
        #for j in range(0,int(fgn_section)):
        for j in range(0,N2):
            Hurst_fgn = random.uniform(0,1) #generates random number between 0 and 1 (random Hurst value)
            Hurst_expected[i][j] = Hurst_fgn
            Sig_class_expected[i][j] = 0 #fGn class
            #generate fGn signal with random Hurst at each element in first third of matrix
            Signal_matrix[i][j] = fgn(n=N3, hurst = Hurst_fgn, length=1, method='daviesharte')
        for k in range(int(N2/2),N2):
            Hurst_fbm = random.uniform(0.3,1)
            Hurst_expected[i][k] = Hurst_fbm
            Sig_class_expected[i][k] = 2 #fBm class
            #generate fBm signal with random Hurst at each element in middle third of matrix
            Signal_matrix[i][k] = fbm(n=N3+1, hurst = Hurst_fbm, length=1, method='daviesharte')[1:N3+1]
        #rightmost third of matrix (random signals)
       
        
    #Now find the Hurst value and signal class that FracTool calculates on these known signals
    Hurst_actual = np.zeros((N1,N2))
    Sig_class_actual = np.zeros((N1,N2))
    for n in range(N1): 
        for m in range(int(N2)):
            result = FracTool(Signal_matrix[n,m], 2.4)
            #result[0] of fractool is sig class
            #result[1] of fractool is Hurst coeff
            Hurst_actual[n,m] = result[1] 
            Sig_class_actual[n,m] = result[0] #comment out if only interested in Hurst MSE
        
    
    Signal_matrix = []
    Hurst_actual = np.nan_to_num(Hurst_actual)

    summation = 0
    misclassified_total = 0
    #if want to compute MAE of whole matrix, use range(N1) for outer for loop 
    #if want to compute MAE of fGn only, use range(int(fgn_section))
    #if want to compute MAE of fBm only, use range(int(fgn_section),int(fbm_section))
    #if want to compute MAE of random signals only, use range(int(fbm_section), N2)
    no_val = 0
    for x in range(N1):
        for y in range(N2):

           
            #if ((Sig_class_actual[x,y]) == 1):
            #    Hurst_actual[x,y] = 0
            #    Hurst_expected[x,y] = 0
            #    no_val = no_val + 1
                
            if ((Sig_class_actual[x,y] != Sig_class_expected[x,y])):
                misclassified_total = misclassified_total + 1
            
            #if ((Sig_class_actual[x,y]) == 3):
            #    Hurst_actual[x,y] = 0
            #    Hurst_expected[x,y] = 0
            #    no_val = no_val + 1
                
            if ((Hurst_actual[x,y]) == 0):
                Hurst_expected[x,y] = 0

            diff = abs((Hurst_actual[x,y] - Hurst_expected[x,y]))
            summation = summation + diff
    
    Misclassified_Rate = misclassified_total/((N1*N2)-no_val)
    MAE = summation/((N1*N2)-no_val)  #need to divide by number that is included - change if only looking at specific section of matrix
    print(Sig_class_expected)
    print(Sig_class_actual)
    print(Hurst_expected)
    print(Hurst_actual)
    print(no_val)
    return [MAE, Misclassified_Rate]
#function call 
#ex:  N1=33, N2=33, N3 = 1024 generates 999 (33x33) signals of length = 1024 time-points
ORIG_6(30,30,317)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: ComplexWarning: Casting complex values to real discards the imaginary part
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: ComplexWarning: Casting complex values to real discards the imaginary part


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2.
  2. 2. 2. 2. 2. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2.
  2. 2. 2. 2. 2. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2.
  2. 2. 2. 2. 2. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2.
  2. 2. 2. 2. 2. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2.
  2. 2. 2. 2. 2. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2.
  2. 2. 2. 2. 2. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2.
  2. 2. 2. 2. 2. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2.
  2. 2. 2. 2. 2. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2.
  2. 2. 2. 2. 2. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2.
  2. 2. 2. 2. 2. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 

[0.04578240564691382, 0.011111111111111112]

In [53]:
from fbm import fbm, fgn, times
import numpy as np
#from ipynb.fs.defs.FracTool_Testing import FracTool
import sklearn
from sklearn.metrics import mean_absolute_error
import random as random

def ORIG_6(N1,N2,N3):
    '''This function generates a matrix of known Hurst values and calculates the MSE of the expected Hurst and the 
    actual Hurst outputted by FracTool. The leftmost third of the matrix are fGn signals, the middle third are fBm 
    signals, and the last third are random arrays. 
    N1 = number of rows in matrix
    N2 = number of columns in matrix
    N3 = number of time-points in signal (3rd dimension in matrix)
    '''
    
    #first generate an matrix with known Hurst values and signal class
    
    #create arrays of expected Hurst and class (if interested in class as well)
    Hurst_expected = np.zeros((N1,N2))
    Sig_class_expected = np.zeros((N1,N2))
    Signal_matrix = np.zeros((N1,N2,N3))
    fgn_section = np.floor(N2/2) #end of leftmost third of columns of matrix
   
    for i in range(N1):
        #leftmost third of matrix (fGn signals)
        #for j in range(0,int(fgn_section)):
        for j in range(0,N2):
            Hurst_fgn = random.uniform(0,1) #generates random number between 0 and 1 (random Hurst value)
            Hurst_expected[i][j] = Hurst_fgn
            Sig_class_expected[i][j] = 0 #fGn class
            #generate fGn signal with random Hurst at each element in first third of matrix
            Signal_matrix[i][j] = fgn(n=N3, hurst = Hurst_fgn, length=1, method='daviesharte')
        #for k in range(int(N2/2),N2):
        #    Hurst_fbm = random.uniform(0.3,1)
        #    Hurst_expected[i][k] = Hurst_fbm
        #    Sig_class_expected[i][k] = 2 #fBm class
            #generate fBm signal with random Hurst at each element in middle third of matrix
        #    Signal_matrix[i][k] = fbm(n=N3+1, hurst = Hurst_fbm, length=1, method='daviesharte')[1:N3+1]
        #rightmost third of matrix (random signals)
       
        
    #Now find the Hurst value and signal class that FracTool calculates on these known signals
    Hurst_actual = np.zeros((N1,N2))
    Sig_class_actual = np.zeros((N1,N2))
    for n in range(N1): 
        for m in range(int(N2)):
            result = FracTool(Signal_matrix[n,m])
            #result[0] of fractool is sig class
            #result[1] of fractool is Hurst coeff
            Hurst_actual[n,m] = result[1] 
            Sig_class_actual[n,m] = result[0] #comment out if only interested in Hurst MSE
        
    
    Signal_matrix = []
    Hurst_actual = np.nan_to_num(Hurst_actual)

    summation = 0
    misclassified_total = 0
    #if want to compute MAE of whole matrix, use range(N1) for outer for loop 
    #if want to compute MAE of fGn only, use range(int(fgn_section))
    #if want to compute MAE of fBm only, use range(int(fgn_section),int(fbm_section))
    #if want to compute MAE of random signals only, use range(int(fbm_section), N2)
    no_val = 0
    for x in range(N1):
        for y in range(N2):

           
            #if ((Sig_class_actual[x,y]) == 1):
            #    Hurst_actual[x,y] = 0
            #    Hurst_expected[x,y] = 0
            #    no_val = no_val + 1
                
            if ((Sig_class_actual[x,y] != Sig_class_expected[x,y])):
                misclassified_total = misclassified_total + 1
            
            #if ((Sig_class_actual[x,y]) == 3):
            #    Hurst_actual[x,y] = 0
            #    Hurst_expected[x,y] = 0
            #    no_val = no_val + 1
                
            if ((Hurst_actual[x,y]) == 0):
                Hurst_expected[x,y] = 0

            diff = abs((Hurst_actual[x,y] - Hurst_expected[x,y]))
            summation = summation + diff
    
    Misclassified_Rate = misclassified_total/((N1*N2)-no_val)
    MAE = summation/((N1*N2)-no_val)  #need to divide by number that is included - change if only looking at specific section of matrix
    print(Sig_class_expected)
    print(Sig_class_actual)
    print(Hurst_expected)
    print(Hurst_actual)
    print(no_val)
    return [MAE, Misclassified_Rate]
#function call 
#ex:  N1=33, N2=33, N3 = 1024 generates 999 (33x33) signals of length = 1024 time-points
ORIG_6(32,32,2000)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: ComplexWarning: Casting complex values to real discards the imaginary part
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: ComplexWarning: Casting complex values to real discards the imaginary part


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 2. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0.2331953  0.1834504  0.74096521 ... 0.         0.64607729 0.20414425]
 [0.63470764 0.9584165  0.08828386 ... 0.17392495 0.86094254 0.82646133]
 [0.9023522  0.12723685 0.99768806 ... 0.03614556 0.91709537 0.71037826]
 ...
 [0.97834972 0.95890015 0.7448319  ... 0.12100877 0.02962234 0.36970185]
 [0.44681378 0.65663089 0.82839286 ... 0.85068236 0.04716949 0.71276117]
 [0.83167496 0.89160176 0.57266445 ... 0.27091157 0.58265308 0.66720765]]
[[ 0.31014026  0.25574155  0.70497955 ...  0.          0.69714951
   0.07979867]
 [ 0.67548738  0.83598681 -0.05230941 ...  0.12346207  0.84383757
   0.75997496]
 [ 0.88960326  0.12831843  0.94137606 ... -0.1513027   0.85724776
   0.58792926]
 

[0.05589880524655445, 0.0126953125]